In [9]:
from __future__ import absolute_import, division, print_function
from builtins import super

import os

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
%matplotlib inline

_Note: all the classes below have the input dimensions (and the input, come to think of it) hardwired into the code. Not nice, but oh well._

### Scaffolding

In [154]:
class ML(object):
    def __init__(self, name='ML'):
        self.name = name
        self.graph = tf.Graph()
        
        with self.graph.as_default():
            with tf.name_scope('model'):
                self.initialize_graph()
            with tf.name_scope('global_ops'):
                self.init = tf.initialize_all_variables()
            
    def initialize_graph(self):
        """Already in graph scope."""
        raise NotImplementedError('Method initialize_graph() must be implemented.')
    
    def inference(self, X):
        """Compute inference model over data X and return the result."""
        raise NotImplementedError('Method inference(X) must be implemented.')
    
    def loss(self, X, Y):
        """Compute loss over training data X and expected outputs Y."""
        raise NotImplementedError('Method loss(X, Y) must be implemented.')
    
    def inputs(self):
        """Read/generate input training data X and expected outputs Y."""
        raise NotImplementedError('Method inputs() must be implemented.')
    
    def train(self, total_loss):
        """Train / adjust model parameters according to computed total loss."""
        raise NotImplementedError('Method train(total_loss) must be implemented.')
        
    def evaluate(self, sess, X, Y):
        """Evaluate the resulting trained model."""
        raise NotImplementedError('Method evaluate(sess, X, Y) must be implemented.')
        
    def run_training(self, training_steps, print_every=10, save_every=1000):
        save_dir = os.path.join('saves', self.name)
        
        with self.graph.as_default():
            X, Y = self.inputs()
        
            total_loss = self.loss(X, Y)
            train_op = self.train(total_loss)
            
            # Create a saver.
            saver = tf.train.Saver()
                
        sess = tf.Session(graph=self.graph)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        
        # Load the latest training checkpoint, if it exists
        ckpt = tf.train.get_checkpoint_state(save_dir)
        if ckpt and ckpt.model_checkpoint_path:
            # Restores from checkpoint
            saver.restore(sess, ckpt.model_checkpoint_path)
            initial_step = int(ckpt.model_checkpoint_path.rsplit('-', 1)[1])
        else:
            sess.run(self.init)
            initial_step = 0
        
        # The actual training loop
        for step in range(initial_step, training_steps):
            sess.run([train_op])
            
            # for debugging and learning purposes, see how the loss gets
            # decremented thru training steps
            if step % print_every == 0:
                print('loss: ', sess.run(total_loss))
            
            # Model checkpoint
            if step % save_every == 0 and step > 0:
                print('Saving in...')
                saver.save(sess, save_dir, global_step=step)
                
        print('Saving out...')
        saver.save(sess, save_dir, global_step=training_steps)
        
        self.evaluate(sess, X, Y)
        coord.request_stop()
        coord.join(threads)
        sess.close()
     
    @staticmethod
    def read_csv(batch_size, file_name, record_defaults):
        filename_queue = tf.train.string_input_producer([file_name])
        reader = tf.TextLineReader(skip_header_lines=1)
        key, value = reader.read(filename_queue)
        
        # decode_csv will convert a Tensor from type string (the text line) in
        # a tuple of tensor columns with the specified defaults, which also
        # sets the data type for each column
        decoded = tf.decode_csv(value, record_defaults=record_defaults)
        
        # batch actually reads the file and loads "batch_size" rows in a single tensor
        return tf.train.shuffle_batch(decoded,
                                      batch_size=batch_size,
                                      capacity=batch_size * 50,
                                      min_after_dequeue=batch_size)

#### Linear regression

In [152]:
class LinearRegression(ML):
    def __init__(self, name='LR'):
        super().__init__(name=name)
        
    def initialize_graph(self):
        # initialize variables/model parameters
        self.W = tf.Variable(tf.zeros([2, 1]), name="weights")
        self.b = tf.Variable(0., name="bias")
        
    def inference(self, X):
        return tf.matmul(X, self.W) + self.b
    
    def loss(self, X, Y):
        beta = 0.001
        Y_predicted = self.inference(X)
        return (tf.reduce_sum(tf.squared_difference(Y, Y_predicted))) #+
                #beta * (tf.nn.l2_loss(self.W) + tf.nn.l2_loss(self.b)))
    
    def inputs(self):
        weight_age = [[84, 46], [73, 20], [65, 52], [70, 30], [76, 57],
                      [69, 25], [63, 28], [72, 36], [79, 57], [75, 44],
                      [27, 24], [89, 31], [65, 52], [57, 23], [59, 60],
                      [69, 48], [60, 34], [79, 51], [75, 50], [82, 34],
                      [59, 46], [67, 23], [85, 37], [55, 40], [63, 30]]
        blood_fat_content = [354, 190, 405, 263, 451,
                             302, 288, 385, 402, 365,
                             209, 290, 346, 254, 395,
                             434, 220, 374, 308, 220,
                             311, 181, 274, 303, 244]
        return tf.to_float(weight_age), tf.to_float(blood_fat_content)

    def train(self, total_loss):
        learning_rate = 0.0000001
        return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)
        
    
    def evaluate(self, sess, X, Y):
        print(sess.run(self.inference([[80., 23.]])))
        print(sess.run(self.inference([[65., 23.]])))
        print(sess.run(self.inference([[57., 23.]])))

In [153]:
lr = LinearRegression()
lr.run_training(100000, print_every=10000, save_every=100000)

loss:  7.60877e+06
loss:  5.22567e+06
loss:  5.11695e+06
loss:  5.01565e+06
loss:  4.92128e+06
loss:  4.83335e+06
loss:  4.75144e+06
loss:  4.67511e+06
loss:  4.60401e+06
loss:  4.53777e+06
Saving out...
[[ 315.59927368]]
[[ 278.35565186]]
[[ 258.49240112]]


#### Logistic Regression

In [167]:
class LogisticRegression(ML):
    def __init__(self, name='LR'):
        super().__init__(name=name)
        
    def initialize_graph(self):
        # initialize variables/model parameters
        self.W = tf.Variable(tf.zeros([5, 1]), name="weights")
        self.b = tf.Variable(0., name="bias")
        
    def combine_inputs(self, X):
        return tf.matmul(X, self.W) + self.b
        
    def inference(self, X):
        return tf.sigmoid(self.combine_inputs(X))
    
    def loss(self, X, Y):
        beta = 0.001
        return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(self.combine_inputs(X), Y))
    
    def inputs(self):
        (passenger_id, survived, pclass, name, sex, age,
         sibsp, parch, ticket, fare, cabin, embarked) = self.read_csv(
            100, 'data/titanic_train.csv',
            [[0.0], [0.0], [0], [""], [""], [0.0],
             [0.0], [0.0], [""], [0.0], [""], [""]])
        
        # convert categorical data
        is_first_class = tf.to_float(tf.equal(pclass, [1]))
        is_second_class = tf.to_float(tf.equal(pclass, [2]))
        is_third_class = tf.to_float(tf.equal(pclass, [3]))
        gender = tf.to_float(tf.equal(sex, ["female"]))
        
        # Finally we pack all the features in a single matrix;
        # We then transpose to have a matrix with one example per row and one
        # feature per column.
        features = tf.transpose(tf.pack([is_first_class, is_second_class,
                                         is_third_class, gender, age]))
        survived = tf.reshape(survived, [100, 1])
        return features, survived

    def train(self, total_loss):
        learning_rate = 0.01
        return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)
        
    def evaluate(self, sess, X, Y):
        # Accuracy: how many correct predictions on average?
        # We have to convert the bool values to to float (T/F -> 1/0)
        with self.graph.as_default():
            predicted = tf.cast(self.inference(X) > 0.5, tf.float32)
            print(sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32))))

In [170]:
lore = LogisticRegression()
lore.run_training(20000, print_every=2000, save_every=10000)

loss:  0.625054
loss:  0.587274
loss:  0.454687
loss:  0.46095
loss:  0.406245
loss:  0.518605
Saving in...
loss:  0.423056
loss:  0.508384
loss:  0.415439
loss:  0.414944
Saving out...
0.77


#### Multinomial Logistic Regression

In [203]:
class MultinomialLogisticRegression(ML):
    def __init__(self, name='LR'):
        super().__init__(name=name)
        
    def initialize_graph(self):
        # initialize variables/model parameters
        self.W = tf.Variable(tf.zeros([4, 3]), name="weights")
        self.b = tf.Variable(tf.zeros([3]), name="bias")
        
    def combine_inputs(self, X):
        return tf.matmul(tf.cast(X, tf.float32), self.W) + self.b
        
    def inference(self, X):
        return tf.nn.softmax(self.combine_inputs(X))
    
    def loss(self, X, Y):
        return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            self.combine_inputs(X), Y))
    
    def inputs(self):
        sepal_length, sepal_width, petal_length, petal_width, label =\
            self.read_csv(50, "./data/iris.data", [[0.0], [0.0], [0.0], [0.0], [""]])

        # Convert class names to a 0 based class index.
        # The last to_int32 is because argmax returns int64
        label_number = tf.to_int32(tf.argmax(tf.to_int32(tf.pack([
            tf.equal(label, ["Iris-setosa"]),
            tf.equal(label, ["Iris-versicolor"]),
            tf.equal(label, ["Iris-virginica"])])), 0))
        
        # Pack all the features that we care about in a single matrix;
        # We then transpose to have a matrix with one example per row and one
        # feature per column.
        features = tf.transpose(tf.pack([sepal_length, sepal_width, petal_length, petal_width]))
        return features, label_number
        
    def train(self, total_loss):
        learning_rate = 0.01
        return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)
        
    def evaluate(self, sess, X, Y):
        # Accuracy: how many correct predictions on average?
        # We have to convert the bool values to to float (T/F -> 1/0)
        with self.graph.as_default():
            predicted = tf.cast(tf.argmax(self.inference(X), 1), tf.int32)
            print(sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32))))

In [205]:
mlr = MultinomialLogisticRegression()
mlr.run_training(15000, print_every=1000, save_every=10000)

loss:  1.14878
loss:  0.374805
loss:  0.32957
loss:  0.21796
loss:  0.20724
loss:  0.128124
loss:  0.139673
loss:  0.125353
loss:  0.163467
loss:  0.175137
loss:  0.168669
Saving in...
loss:  0.154647
loss:  0.144735
loss:  0.142105
loss:  0.128798
Saving out...
0.98
